[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/21--no-sql-database-mongo/05--document-database_mongodb--movies.ipynb)

In [1]:
!sudo apt install mongodb
!sudo service mongodb start

Unable to locate an executable at "/Library/Java/JavaVirtualMachines/adoptopenjdk-11.jdk/Contents/Home/bin/apt" (-1)
sudo: service: command not found


## datasample

In [2]:
!mkdir -p var
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson -O var/video_movies.bson
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movieDetails.bson -O var/video_movieDetails
!mongorestore --db datascience --drop --collection movies  
!mongorestore --db datascience --drop --collection films video.bson"    

--2021-11-20 11:23:39--  https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson [following]
--2021-11-20 11:23:39--  https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343355 (335K) [application/octet-stream]
Saving to: ‘var/video_movies.bson’

var/video_movies.bs 10

Queries

```json
{
     "_id" : ObjectId("5b107bec1d2952d0da904dd7"),
     "title" : "Titan A.E.",
     "year" : 2000,
     "rated" : "PG",
     "runtime" : 94,
     "countries" : [
             "USA"
     ],
     "genres" : [
             "Animation",
             "Action",
             "Adventure"
     ],
     "director" : "Don Bluth, Gary Goldman, Art Vitello",
     "writers" : [
             "Hans Bauer",
             "Randall McCormick",
             "Ben Edlund",
             "John August",
             "Joss Whedon"
     ],
     "actors" : [
             "Matt Damon",
             "Bill Pullman",
             "John Leguizamo",
             "Nathan Lane"
     ],
     "plot" : "A young man learns that he has to find a hidden Earth ship before an enemy alien species does in order to secure the survival of humanity.",
     "poster" : "http://ia.media-imdb.com/images/M/MV5BMjE0NTU0ODg4NV5BMl5BanBnXkFtZTcwNzY3MTQyMQ@@._V1_SX300.jpg",
     "imdb" : {
             "id" : "tt0120913",
             "rating" : 6.6,
             "votes" : 50875
     },
     "tomato" : {
             "meter" : 52,
             "image" : "rotten",
             "rating" : 5.7,
             "reviews" : 99,
             "fresh" : 51,
             "consensus" : "Great visuals, but the story feels like a cut-and-paste job of other sci-fi movies.",
             "userMeter" : 60,
             "userRating" : 3.2,
             "userReviews" : 69055
     },
     "metacritic" : 48,
     "awards" : {
             "wins" : 1,
             "nominations" : 7,
             "text" : "1 win & 7 nominations."
     },
     "type" : "movie"
}
```

In [29]:
#Connect to database
!pip install pymongo

You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://localhost:27017')

In [31]:
db = client.datascience

Movie titles with rating over 8.5

In [32]:
pp(next(db.movies.find(
    {},
    {"title": 1, "imdb.rating": 1}
)))

{'_id': ObjectId('5692a13b24de1e0ce2dfcea8'),
 'imdb': {'rating': 7.2},
 'title': 'Planet Terror'}


Operators
---------
distinguished with $

* Inequality
    - \$gt
    - \$lt
    - \$gte
    - \$lte
    - \$ne


In [33]:
pp(next(db.movies.find(
    {"imdb.rating": {"$gt": 8.9}},
    {"title": 1, "imdb.rating": 1}
)))

{'_id': ObjectId('5692a27f24de1e0ce2dfd712'),
 'imdb': {'rating': 9},
 'title': 'Gamechangers Ep. 3: A Legend in the Booth'}


In [34]:
pp(next(db.movies.find(
    {"imdb.rating": {"$gt": 8.9, "$lt": 9.5}},
    {"title": 1, "imdb.rating": 1}
)))

{'_id': ObjectId('5692a27f24de1e0ce2dfd712'),
 'imdb': {'rating': 9},
 'title': 'Gamechangers Ep. 3: A Legend in the Booth'}


how many movies are over 9 in our collection

In [35]:
movies_count = 0

for movie in db.movies.find(
    {"imdb.rating": {"$gt": 8.9, "$lt": 9.5}},
    {"title": 1, "imdb.rating": 1}
):
    movies_count += 1

print("There is {} movies rated over 9.0".format(movies_count))

There is 8 movies rated over 9.0


exists field

In [36]:
def count_results(cursor):
    count = 0
    for item in cursor:
        count += 1
    return count

In [37]:
count_results(db.movies.find(
    {"tomato": {"$exists": 1}},
    {"title": 1, "imdb.rating": 1}
))

376

In [38]:
count_results(db.movies.find(
    {"tomato": {"$exists": 0}},
    {"title": 1, "imdb.rating": 1}
))

1919

regex Operator
--------------


In [39]:
count_results(db.movies.find(
    {"title": {"$regex": "super"}},
    {"title": 1, "imdb.rating": 1}
))

0

In [40]:
count_results(db.movies.find(
    {"title": {"$regex": "[Ss]uper"}},
    {"title": 1, "imdb.rating": 1}
))

6

In [41]:
count_results(db.movies.find(
    {"title": {"$regex": "super", "$options": 'i'}},
    {"title": 1, "imdb.rating": 1}
))

6

In [42]:
pp(list(map(lambda x: x['title'],db.movieDetails.find(
    {"title": {"$regex": "super|hero", "$options": 'i'}},
    {"title": 1, "imdb.rating": 1}
))))

[]


In [43]:
count_results(db.movies.find(
    {"imdb.rating": {"$gt": 9}, "year": {"$in": list(range(2008, 2019, 1))}},
    {"title": 1, "imdb.rating": 1}
))

0

Sadly there is no movies rated over 9 after 2009

In [44]:
db.movies.find(
    {"title": {"$regex": "super|hero", "$options": 'i'}},
    {"title": 1, "imdb.rating": 1}
).count()

/root/workspace/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


9